# 🚂 YOLOv11 Training Pipeline

This notebook implements the training pipeline for a custom object detection model using **YOLOv11**.

**Objective:** Train a large-scale model (`yolo11x`) to detect animals in drone footage.
**Output:** A trained `.pt` weight file that can be used for tracking and inference.

## 1. Dataset & File Structure Requirements

Before running this notebook, ensure your Google Drive is organized correctly. YOLO requires a specific folder structure to find images and labels.

**1. Directory Structure:**
Your `datasets` folder should look like this:
```text
/content/drive/MyDrive/datasets/
│
├── TestData.yaml          <-- The config file pointing to train/val folders
├── train/
│   ├── images/            <-- Training images (.jpg/.png)
│   └── labels/            <-- Training labels (.txt)
├── val/
│   ├── images/            <-- Validation images
│   └── labels/            <-- Validation labels
└── test/
    ├── images/
    └── labels/

In [ ]:
### **Cell 3: [Code] - Installation & Setup**
# @title 2. Install Dependencies & Mount Drive
# We install the ultralytics package which contains the YOLOv11 framework.
%pip install ultralytics -q

import torch
import os
from ultralytics import YOLO
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Check for GPU
if torch.cuda.is_available():
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU detected. Training will be extremely slow. Go to Runtime > Change runtime type > T4 GPU.")

In [ ]:
# @title 3. Path Configuration
# --- USER CONFIGURATION ---
# Define the path to your dataset folder in Drive.
# This folder must contain your 'TestData.yaml'.
WORKING_DIR = '/content/drive/MyDrive/datasets'

# Set the working directory
if os.path.exists(WORKING_DIR):
    os.chdir(WORKING_DIR)
    print(f"✅ Working Directory set to: {os.getcwd()}")

    # Check for the YAML file
    if os.path.exists("TestData.yaml"):
        print("✅ Found TestData.yaml")
    else:
        print("❌ Warning: TestData.yaml not found in this directory. Training will fail.")
else:
    print(f"❌ Error: Path not found: {WORKING_DIR}")
    print("Please update the WORKING_DIR variable.")

## 4. Model Training

We will now train the **YOLOv11 Extra Large (`yolo11x`)** model.

**Hyperparameters:**
* **Model:** `yolo11x.pt` (Highest accuracy, slowest speed).
* **Epochs:** `150` (Total training iterations).
* **Batch Size:** `8` (Optimized for Colab GPU memory. If you get "Out of Memory" errors, reduce this to 4).
* **Image Size:** `640` pixels.
* **Learning Rate:** Initial `0.01`, Final `0.1` decay factor.
* **Optimizer:** `auto` (YOLO usually selects SGD or AdamW automatically).

In [ ]:
# @title Start Training
# Clear GPU cache to ensure maximum memory availability
torch.cuda.empty_cache()

# 1. Load the Pre-trained Model
print("⬇️ Loading YOLOv11x pre-trained weights...")
model = YOLO("yolo11x.pt")

# 2. Define Training Arguments
project_name = "runs/detect"
experiment_name = "train_xl_v1"

print("🚀 Starting Training Loop...")
try:
    results = model.train(
        data="TestData.yaml",   # Path to dataset config
        epochs=150,             # Number of epochs
        imgsz=640,              # Image size
        batch=8,                # Batch size
        lr0=0.01,               # Initial learning rate
        lrf=0.1,                # Final learning rate
        weight_decay=0.0005,    # Regularization
        save_period=10,         # Save checkpoint every 10 epochs
        project=project_name,   # Output folder
        name=experiment_name,   # Experiment folder
        exist_ok=True,          # Overwrite if exists
        verbose=True
    )
    print(f"\n✅ Training Complete. Best model saved at: {os.path.join(WORKING_DIR, project_name, experiment_name, 'weights/best.pt')}")

except Exception as e:
    print(f"\n❌ Training Failed: {e}")

## 5. View Results

After training, the results (including the trained model and performance graphs) are saved in the `runs/detect/train_xl_v1` folder.

**Key Files:**
* `weights/best.pt`: The model with the highest validation accuracy. **Use this for inference.**
* `weights/last.pt`: The model state at the very last epoch.
* `results.csv`: Detailed metrics for every epoch.
* `results.png`: Graphs of loss, precision, and recall over time.

In [ ]:
# @title Display Training Performance
from IPython.display import Image, display

# Path to the results graph
results_path = f"{project_name}/{experiment_name}/results.png"

if os.path.exists(results_path):
    print("Displaying Training Metrics:")
    display(Image(filename=results_path, width=800))
else:
    print("Results graph not found. (Training might not have finished yet).")